In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split

train = pd.read_csv('house-prices-advanced-regression-techniques\\train.csv', usecols=[1, 17, 18, 19, 43, 44, 46, 49, 50, 70, 77, 80])
test  = pd.read_csv('house-prices-advanced-regression-techniques\\test.csv', usecols=[1, 17, 18, 19, 43, 44, 46, 49, 50, 70, 77])

In [2]:
kfolds = KFold(n_splits=10, shuffle=True, random_state=420)
X_train, y_train = train.loc[:, train.columns != 'SalePrice'], np.log(train['SalePrice'])

In [3]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))
def rmse(model, X=X_train,y=y_train):
    rmse = np.sqrt(-cross_val_score(model, X, y,scoring="neg_mean_squared_error",cv=kfolds))
    return (rmse)
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse= np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [4]:
lasso = make_pipeline(RobustScaler(), Lasso())
ENet = make_pipeline(RobustScaler(), ElasticNet())
KRR = KernelRidge()
GBoost = GradientBoostingRegressor()

lasso_model = lasso.fit(X_train,y_train)
ridge_model = ridge.fit(X_train,y_train)
elasticnet_model = elasticnet.fit(X_train,y_train)
randomforest_model = rf.fit(X_train,y_train)
lr_model = lr.fit(X_train,y_train)
tree_model =tree.fit(X_train,y_train)

In [5]:
def blend_models(X):
    return (
        (.1 * lasso_model.predict(X)) + \
        (.1 * ridge_model.predict(X)) + \
        (.1 * elasticnet_model.predict(X)) + \
        (.5 * randomforest_model.predict(X)) + \
        (.1 * lr_model.predict(X)) + \
        (.1 * tree_model.predict(X))
           )

In [6]:
averaged_models = AveragingModels(models = (ENet, GBoost, KRR, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 0.2068 (0.0094)



In [9]:
averaged_models.fit(train.values, y_train)
test['Prediction']=averaged_models.predict(test)

ValueError: operands could not be broadcast together with shapes (1459,11) (12,) (1459,11) 